In [1]:
import numpy as np

# generate data

In [2]:
# In real world, you cannot learn how the data was generated. So do not rely on this function when coding your lab.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

In [ ]:
# example
x, y, mr = generate_data(5, 100)
x[:5], y[:5]

# write your model class

In [39]:
# SMO method
import time as t
class SVM1:
    def __init__(self, dim, tol = 1e-3, C=1.0):
        """"""
        self.dim = dim
        self.w = np.zeros((1,dim))
        self.b = 0.0#1.0
        self.tol = tol
        self.C = C
        self.max_iter = 10 # just for stop in test
        self.train_time = 0

    def fit(self, X, y):
        """ X is m*dim 's ndarray, y is m ndarray """
        err_msg = "The data must be of {}D, but got dimension: {}".format(self.dim, X.shape[1])
        assert X.shape[1] == self.dim, err_msg
        start = t.time()
        #cost = []

        #y = np.mat(y)   # m*1 ndarray
        alpha = np.zeros((X.shape[0], 1))
        iter = 0
        while(iter < self.max_iter):
            #a1 = self.check_KKT(X, y, alpha) #self.get_alpha1()
            for a1 in range(X.shape[0]):
                e1 = self.get_err(X, y, a1, self.b, alpha)
                if((y[a1]*e1 < -self.tol) and (alpha[a1]<self.C)) or \
                    ((y[a1]*e1 > self.tol) and (alpha[a1]>0)):
                    a2 = self.get_alpha2_2(X.shape[0], a1)
                    #a2 = self.get_alpha2_1(X, y, a1)
                    tmp_a1 = alpha[a1][0]
                    tmp_a2 = alpha[a2][0]

                    # 优化问题矩形可行域
                    if(y[a1]!=y[a2]):
                        L = max(0.0, alpha[a2] - alpha[a1])
                        U = min(self.C, self.C + alpha[a2] - alpha[a1])# 1自定义
                    else:
                        L = max(0.0, alpha[a2] + alpha[a1] -self.C)
                        U = min(self.C, alpha[a2] + alpha[a1])
                    if(L==U):
                        continue

                    # 提前做一些运算，可以省去大量矩阵计算
                    e2 = self.get_err(X, y, a2, self.b, alpha)
                    K11 = np.dot(X[a1], X[a1].T)
                    K12 = np.dot(X[a1], X[a2].T)
                    K22 = np.dot(X[a2], X[a2].T)
                    
                    A = K11 + K22 - 2*K12
                    if(A<0):
                        print("err")
                        continue
                    alpha[a2][0] = tmp_a2 + y[a2] * (e1 - e2) / A
                    if(alpha[a2] > U):
                        alpha[a2] = U                   
                    if(alpha[a2] < L):
                        alpha[a2] = L
                    alpha[a1][0] = tmp_a1 - y[a1]*y[a2]*(alpha[a2] - tmp_a2)

                    #alpha = self.update(X, y, alpha, a1, a2 ,tmp_1, tmp_2) # update alpha, w and b
                    b1 = self.b - e1 + y[a1] * (tmp_a1 - alpha[a1]) * K11 + y[a2] * (tmp_a2 - alpha[a2]) * K12
                    b2 = self.b - e2 + y[a1] * (tmp_a1 - alpha[a1]) * K12 + y[a2] * (tmp_a2 - alpha[a2]) * K22
                    if (0 < alpha[a1]) and (alpha[a1] < self.C):
                        self.b = b1
                    elif (0 < alpha[a1]) and (alpha[a1] < self.C):
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2

                    #alpha[alpha<1e-5] = 0
                    self.w = np.dot((y * alpha).T, X)     
                    #cost.append(self.cost_func(X, y, self.w, self.b))
            iter += 1      
        
        end = t.time()
        self.train_time = end - start
        return  alpha#, cost

    #absorbed    
    def get_alpha2_1(self, X, y, a):
        y_hat = self.predict(X)
        Err = y - y_hat #debug
        Err2a = np.absolute(Err - Err[a])
        
        if((Err2a > self.tol).any()):
            return np.argmax(Err2a)
        else: #偷个懒
            if(a < 1):
                return 1
            return np.random.randint(0,a)
    
    def get_alpha2_2(self, n, a):
        b = np.random.randint(n)
        while(b==a):
            b = np.random.randint(n)
        return b
    #absorbed
    def get_alpha2_3(self, X, y, a):
        '''选取距离a较远且alpha[a]!=0的点'''

    def get_err(self, X, y, i, b, alphas):
        """返回X的第a条数据的预测偏差"""
        err = np.dot(np.dot((y*alphas).T, X), X[i].T) + b - y[i]
        #print(err)
        return err

    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        X is n*dim 's array
        """
        y_hat = np.dot(X, self.w.T) + self.b
        y_hat[y_hat>0] = 1.0#
        y_hat[y_hat<0] = -1.0#
        return y_hat

    def cost(self, X, y, w, b):
        """The cost function"""
        result = 0
        y_hat = -(np.dot(X, w.T) + b) + 1
        y_hat[y_hat<0] = 0
        result = sum(y_hat)/X.shape[0]
        result = self.C * result + (np.linalg.norm(w) ** 2) / 2
        return result

In [3]:
# SGD/ mini-batch
import time as t
class SVM2:
    def __init__(self, dim, C=10, lamda=1, max_iter=10):
        """
        You can add some other parameters, which I think is not necessary
        """
        self.dim = dim
        self.C = C
        self.lamda = lamda
        self.max_iter = max_iter
        self.w = np.zeros((1, dim))
        self.b = 0

    def fit(self, X, y, lr=1e-4, tol=1e-4):
        """
        Fit the coefficients via your methods
        """
        w = np.zeros((1, self.dim))
        b=0
        #cost = []
        start = t.time()

        #arr = [i for i in range(X.shape[0])]
        for j in range(self.max_iter):
            arr=np.array([i for i in range(X.shape[0])])
            np.random.shuffle(arr)
            for i in arr:
                if y[i] * (X[i].dot(w.T)) >= 1:
                    gradient = 0
                    grad_b = 0
                else :
                    gradient = -y[i] * X[i]
                    grad_b = -y[i]
                gradient = w + (self.C * gradient)
                grad_b = self.C * grad_b
                w = w - lr * gradient
                b = b - lr * grad_b
                #cost.append(self.cost(X, y, w, b))

            if np.linalg.norm(lr * gradient) < tol:
                break
            #arr = np.random.shuffle(arr)
            #if(np.linalg.norm(lr * grad) < tol):
            #    break        
        self.w = w
        self.b = b

        end = t.time()
        self.train_time = end - start
        return #cost

    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        X is n*dim 's array
        """
        y_hat = np.dot(X, self.w.T) + self.b
        y_hat[y_hat>0] = 1.0#
        y_hat[y_hat<-0] = -1.0#
        return y_hat

    def cost(self, X, y, w, b):
        """The cost function"""
        result = 0
        y_hat = -(np.dot(X, w.T) + b) + 1
        y_hat[y_hat<0] = 0
        result = sum(y_hat)/X.shape[0]
        result = self.C * result + (np.linalg.norm(w) ** 2) / 2
        return result

# construct and train your models

In [67]:
# generate data
dim = 20
size = 30000
train_ratio = 0.9
X_data, y_data, mislabel = generate_data(dim, size) 

# split data
rows = np.arange(size)
train_num = int(size * train_ratio)
train_X = X_data[rows[: train_num]]
test_X = X_data[rows[train_num:]]
train_y = y_data[rows[: train_num]]
test_y = y_data[rows[train_num:]]

In [ ]:
# constrcut model and train (remember record time)
model1 = SVM1(dim=dim) 
alpha = model1.fit(train_X, train_y)

y_hat = model1.predict(train_X)
print("training accuarcy: ",1 - sum(np.absolute(train_y - y_hat))[0]/2/train_num)
print("training time: ", model1.train_time, "s")
print("支持向量数： ", sum(alpha>1e-5))

In [75]:
model2 = SVM2(dim=dim) 
model2.fit(train_X, train_y) #cost = 

y_hat = model2.predict(train_X)
print("training accuarcy: ",1 - sum(np.absolute(train_y - y_hat))[0]/2/train_num)
print("training time: ", model2.train_time, "s")

training accuarcy:  0.910962962962963
training time:  0.20905756950378418 s


# predict and compare your results

In [ ]:
# method one: SMO
# make prediction
y_pred1 = model1.predict(test_X)

# compared with answer
TP = sum((test_y + 2*y_pred1) == 3)
FN = sum((test_y + 2*y_pred1) == -1)
TN = sum((test_y + 2*y_pred1) == -3)
FP = sum((test_y + 2*y_pred1) == 1)

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (FP + TP)
recall = TP / (TP + FN)

print("mislabel: ", mislabel, "accuarcy: ",accuracy, "precision: ", precision, "recall: ", recall)

In [77]:
# method two: SGD
# make prediction
y_pred1 = model2.predict(test_X)

# compared with answer
TP = sum((test_y + 2*y_pred1) == 3)
FN = sum((test_y + 2*y_pred1) == -1)
TN = sum((test_y + 2*y_pred1) == -3)
FP = sum((test_y + 2*y_pred1) == 1)

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (FP + TP)
recall = TP / (TP + FN)

print("mislabel: ", mislabel, "accuarcy: ",accuracy, "precision: ", precision, "recall: ", recall)

mislabel:  0.037766666666666664 accuarcy:  [0.909] precision:  [0.90566038] recall:  [0.91578947]


## 以下不属于Lab2，用于测试```sklearn```的模型验证函数

1. ```metrics``` 下的函数 ```accuracy_score()```

In [7]:
from sklearn.metrics import accuracy_score
#accuracy_score(test_y, y_pred1), accuracy_score(train_y, y_hat)

2. ```sklearn.cross_validation``` 下的 ```train_test_split()```
    
    ```X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_data,train_target,test_size=0.4, random_state=0,stratify=y_train)```
    - train_target：所要划分的样本结果
    - test_size：样本占比，如果是整数的话就是样本的数量
    - random_state：随机数的种子，在需要重复试验的时候，除0以外，相同参数可以保证得到一样的划分。但填0或不填，不会得到重复结果。
    - stratify是为了保持split前类的分布。


In [4]:
# generate data
dim = 20
size = 30000
train_ratio = 0.9
X_data, y_data, mislabel = generate_data(dim, size) 

In [21]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X_data, y_data, random_state=0, train_size=0.9,stratify=y_data)

In [34]:
model2 = SVM2(dim=dim) 
model2.fit(train_X, train_y, tol=1e-3)
y_hat = model2.predict(train_X)
y_pred2 = model2.predict(test_X)

accuracy_score(test_y, y_pred2), accuracy_score(train_y, y_hat)

(0.9043333333333333, 0.9145555555555556)

3. ```model_selection.cross_val_score(estimatoe,X)```
    - estimator: 分类器
    - X：array-like，需要学习的数据，可以是列表或2d数组
    - y：array-like，可选的，默认为None，监督学习中样本特征向量的真实目标值
    - scoring：string,callable or None,可选的，默认为None。
        一个字符or一个scorer可调用对象或函数，须实现scorer(estimator,X,y)
    - cv：int，交叉验证生成器或者一个迭代器，可选的，默认为None，决定交叉验证划分策略，cv的可选项有以下几种：
        1. None：使用默认的3-fold交叉验证
        2. Integer：指定在(Stratified)kfold中使用的“折”的数量
        3. 可以用作交叉验证生成器的一个对象
        4. 一个能够产生train/test划分的迭代器对象

该方法可以自行实现，关键是需要一套完整的评分体系（最好能给多分类结果使用），正确率、准确率、召回率、f1、训练时间等

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(model2,X_data,y_data) # model2 没有score方法，不要运行该代码块